# Exploring Reinforcement Learning by Building a ConnextX Agent
Connect X is a competition hosted by Kaggle, requiring us to build an agent
which is able to win a "Connect 4" game

https://www.kaggle.com/competitions/connectx/overview

In this notebook we want to develop different reinforcement learning
algorithms from scratch in order to build a reasonably strong agent.

The goal is not to place high in the competition, but to gain a deeper insight into the
area of reinforcement learning

# Problem Definition
## Agent & Action Space
The agent needs to perform an action. The action space consists of 7 possible actions, since the agent can select either of the 7 columns to place the bead.

## Reward
Currently, the game context provides a reward of 1 for winning a game and a
reward of 0 for losing a game. Therefore, an action according to the game
context is equal to playing one game.

# How we move forward
We will start with the simplest possible agent. Then we will develop a agent which
is able to beat the previous agent. That way, we can try out different ways of
implementing reinforcement learning and evaluate them by letting two agents play
against each other.

Hopefully, we can come up with an agent which performs reasonably well against
the other agents that are submitted to the competition.

# The first Agent
The first agent, which is also the simplest one, is the random agent. It chooses
a random, non-full column to place its bead. It is also the agent used as an
example in the "Getting Started" Notebook.

In [ ]:
from random import choice


def random_agent(observation, configuration):
    c = configuration
    return choice([c for c in range(configuration.columns) if observation.board[c] == 0])

For testing, let's have two random agents play against each other

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make('connectx', debug=True)

env.run([random_agent, random_agent])
env.render(mode='ipython')

KeyboardInterrupt: 

# The first Idea: The omniscient Agent
As of the time of writing, I don't know a lot about RL. I know about the basics
and about how an aged based on the Markov Decision Process works. Therefore,
the first idea would be to change the scope of the learning process. Instead of
rewarding wins and losses, we would reward individual actions.
Any actions that would bring the agent closer to any win condition (or the
 opponent further away from their win condition) would
yield a positive reward, any other action would yield a negative reward. However,
the markov chain to accomplish this would need to know any possible state of the
game.

## How many states are possible
For my first attempt, I want to know if this idea is even remotely realistic.
Calculating the amount of all possible states is difficult by hand. Therefore
I will simply provide the solution and quote its source:

$$
7.1\cdot10^{13}=71,000,000,000,000
$$
(A Knowledge-based Approach of Connect-Four, Victor Allis, 1988)

So there are about 71 trillion possible states. If we encode a state, we need
to encode at least 42 2-bit values for each of the 3 possible states of one
 position [blue, white, empty] and at least 7 1-bit values for the reward-edge
 to the 7 possible subsequent states. To encode an entire markov chain with all possible states, we would need encode
$$
71,000,000,000,000\cdot\left(42\cdot2+7\right)=6,461,000,000,000,000\:bits=807.6\:TB
$$

## Conclusion
If we had access to a supercomputer, building a omniscient agent might be possible,
but we clearly need to be able to run our agent inside a kaggle notebook. Therefore,
we need to find a different strategy.

# The Second Idea: Pointing the Agent in the right Direction
The next idea doesn't care about the entire state of the game. We can generalize
the goal of the game to "Try to connect 4 and prevent the opponent to connect 4".

So, lets try to reward actions that get the agents closer to the goal using
the following value table:

| State                         | Value |
| ----------------------------- | ----- |
| Connect 1                     | 0/6   |
| Connect 2                     | 2/6   |
| Connect 3                     | 4/6   |
| Connect 4                     | 6/6   |
| Prevent Opponent to Connect 1 | 0/6   |
| Prevent Opponent to Connect 2 | 1/6   |
| Prevent Opponent to Connect 3 | 3/6   |
| Prevent Opponent to Connect 4 | 5/6   |

In that scenario, we reward the agent for winning the game and for preventing
the opponent to win the game. The closer to a win a state is, the higher the
reward, though winning yourself will be rewarded higher than preventing the
opponent to win (if both players have connected 3 disks so far, winning the
game in the current turn is more important than preventing the opponent to win
the game in their next turn)

In [ ]:
def simple_agent(observation, configuration):
    pass